In [50]:
from collections import defaultdict
from functools import reduce
from pathlib import Path
from time import perf_counter
import sys

from IPython.core.display import display
from pandas import CategoricalDtype
import numpy as np
from pyopenms import *
import pandas as pd
import os

In [51]:

class ConsensusMapDF(ConsensusMap):
    def __init__(self):
        super().__init__()

    def get_intensity_df(self):
        labelfree = self.getExperimentType() == "label-free"
        filemeta = self.getColumnHeaders()  # type: dict[int, ColumnHeader]
        labels = list(set([header.label for header in
                           filemeta.values()]))  # TODO could be more efficient. Do we require same channels in all files?
        files = list(set([header.filename for header in filemeta.values()]))
        label_to_idx = {k: v for v, k in enumerate(labels)}
        file_to_idx = {k: v for v, k in enumerate(files)}

        def gen(cmap: ConsensusMap, fun):
            for f in cmap:
                yield from fun(f)

        if not labelfree:
            # TODO write two functions for LF and labelled. One has only one channel, the other has only one file per CF
            def extractRowBlocksChannelWideFileLong(f: ConsensusFeature):
                subfeatures = f.getFeatureList()  # type: list[FeatureHandle]
                filerows = defaultdict(lambda: [0] * len(labels))  # TODO use numpy array?
                for fh in subfeatures:
                    header = filemeta[fh.getMapIndex()]
                    row = filerows[header.filename]
                    row[label_to_idx[header.label]] = fh.getIntensity()
                return (f.getUniqueId(), filerows)

            def extractRowsChannelWideFileLong(f: ConsensusFeature):
                uniqueid, rowdict = extractRowBlocksChannelWideFileLong(f)
                for file, row in rowdict.items():
                    row.append(file)
                    yield tuple([uniqueid] + row)

            if len(labels) == 1:
                labels[0] = "intensity"
            dtypes = [('id', np.dtype('uint64'))] + list(zip(labels, ['f'] * len(labels)))
            dtypes.append(('file', 'U300'))
            # For TMT we know that every feature can only be from one file, since feature = PSM
            #cnt = 0
            #for f in self:
            #    cnt += f.size()

            intyarr = np.fromiter(iter=gen(self, extractRowsChannelWideFileLong), dtype=dtypes, count=self.size())
            return pd.DataFrame(intyarr).set_index('id')
        else:
            # Specialized for LabelFree which has to have only one channel
            def extractRowBlocksChannelLongFileWideLF(f: ConsensusFeature):
                subfeatures = f.getFeatureList()  # type: list[FeatureHandle]
                row = [0.] * len(files)  # TODO use numpy array?
                for fh in subfeatures:
                    header = filemeta[fh.getMapIndex()]
                    row[file_to_idx[header.filename]] = fh.getIntensity()
                yield tuple([f.getUniqueId()] + row)

            dtypes = [('id', np.dtype('uint64'))] + list(zip(files, ['f'] * len(files)))
            # cnt = self.size()*len(files) # TODO for this to work, we would need to fill with NAs for CFs that do not go over all files
            cnt = self.size()

            intyarr = np.fromiter(iter=gen(self, extractRowBlocksChannelLongFileWideLF), dtype=dtypes, count=cnt)
            return pd.DataFrame(intyarr).set_index('id')

    def get_metadata_df(self):
        def gen(cmap: ConsensusMap, fun):
            for f in cmap:
                yield from fun(f)

        def extractMetaData(f: ConsensusFeature):
            # subfeatures = f.getFeatureList()  # type: list[FeatureHandle]
            pep = f.getPeptideIdentifications()  # type: list[PeptideIdentification]
            if len(pep) != 0:
                hits = pep[0].getHits()
                if len(hits) != 0:
                    besthit = hits[0]  # type: PeptideHit
                    # TODO what else
                    yield f.getUniqueId(), besthit.getSequence().toString(), f.getCharge(), f.getRT(), f.getMZ(), f.getQuality()
                else:
                    yield f.getUniqueId(), None, f.getCharge(), f.getRT(), f.getMZ(), f.getQuality()
            else:
                yield f.getUniqueId(), None, f.getCharge(), f.getRT(), f.getMZ(), f.getQuality()

        cnt = self.size()

        mddtypes = [('id', np.dtype('uint64')), ('sequence', 'U200'), ('charge', 'i4'), ('RT', 'f'), ('mz', 'f'),
                    ('quality', 'f')]
        mdarr = np.fromiter(iter=gen(self, extractMetaData), dtype=mddtypes, count=cnt)
        return pd.DataFrame(mdarr).set_index('id')

In [52]:
cmap = ConsensusMapDF()
from urllib.request import urlretrieve
urlretrieve ("https://raw.githubusercontent.com/OpenMS/OpenMS/develop/src/tests/class_tests/openms/data/BSA.consensusXML", "label-free.consensusXML")

ConsensusXMLFile().load("label-free.consensusXML", cmap)
    

In [53]:
display(cmap.get_intensity_df())

,/Users/pfeuffer/git/OpenMS-inference-src/share/OpenMS/examples/FRACTIONS/BSA2_F1.mzML,/Users/pfeuffer/git/OpenMS-inference-src/share/OpenMS/examples/FRACTIONS/BSA3_F2.mzML,/Users/pfeuffer/git/OpenMS-inference-src/share/OpenMS/examples/FRACTIONS/BSA3_F1.mzML,/Users/pfeuffer/git/OpenMS-inference-src/share/OpenMS/examples/FRACTIONS/BSA1_F2.mzML,/Users/pfeuffer/git/OpenMS-inference-src/share/OpenMS/examples/FRACTIONS/BSA2_F2.mzML,/Users/pfeuffer/git/OpenMS-inference-src/share/OpenMS/examples/FRACTIONS/BSA1_F1.mzML
id,,,,,,
18055798904544351710,2.788050e+03,0.0,0.0,0.000000e+00,0.000000e+00,0.0
16751911815002726321,0.000000e+00,0.0,461846.0,0.000000e+00,0.000000e+00,1358150.0
1766075384941176729,2.140300e+05,0.0,104389.0,0.000000e+00,0.000000e+00,0.0
6714187641100376547,3.881570e+06,0.0,0.0,0.000000e+00,0.000000e+00,0.0
300941239321730683,1.103670e+07,0.0,3691860.0,0.000000e+00,0.000000e+00,20567800.0
8470403259047476092,4.102750e+06,0.0,760472.0,0.000000e+00,0.000000e+00,1971500.0
17001643603461665041,1.358120e+07,0.0,0.0,0.000000e+00,0.000000e+00,12925300.0
5658659041765702685,3.474620e+07,0.0,13570600.0,0.000000e+00,0.000000e+00,62024400.0
11003401133233860035,0.000000e+00,0.0,2598460.0,0.000000e+00,0.000000e+00,12406600.0


In [54]:
display(cmap.get_metadata_df())

,sequence,charge,RT,mz,quality
id,,,,,
18055798904544351710,DGDIEAEISR,3,1523.370605,368.843781,-2.958420
16751911815002726321,SHC(Carbamidomethyl)IAEVEK,3,1550.230469,358.174591,4.058410
1766075384941176729,SHCIAEVEK,2,1646.545044,508.247498,4.302580
6714187641100376547,QEPERNEC(Carbamidomethyl)FLSHK,3,1717.691528,558.594849,3.741000
300941239321730683,C(Carbamidomethyl)C(Carbamidomethyl)TESLVNR,2,1726.187988,569.752625,4.410060
8470403259047476092,LC(Carbamidomethyl)VLHEK,2,1726.379639,449.744385,4.092430
17001643603461665041,LC(Carbamidomethyl)VLHEK,3,1727.822021,300.165344,3.820900
5658659041765702685,DDSPDLPK,2,1731.364868,443.711273,4.199150
11003401133233860035,EC(Carbamidomethyl)C(Carbamidomethyl)DKPLLEK,3,1743.927124,431.205536,3.663040


In [55]:
cmap.get_metadata_df().merge(cmap.get_intensity_df(), how='left', on='id') # single table

,sequence,charge,RT,mz,quality,/Users/pfeuffer/git/OpenMS-inference-src/share/OpenMS/examples/FRACTIONS/BSA2_F1.mzML,/Users/pfeuffer/git/OpenMS-inference-src/share/OpenMS/examples/FRACTIONS/BSA3_F2.mzML,/Users/pfeuffer/git/OpenMS-inference-src/share/OpenMS/examples/FRACTIONS/BSA3_F1.mzML,/Users/pfeuffer/git/OpenMS-inference-src/share/OpenMS/examples/FRACTIONS/BSA1_F2.mzML,/Users/pfeuffer/git/OpenMS-inference-src/share/OpenMS/examples/FRACTIONS/BSA2_F2.mzML,/Users/pfeuffer/git/OpenMS-inference-src/share/OpenMS/examples/FRACTIONS/BSA1_F1.mzML
id,,,,,,,,,,,
18055798904544351710,DGDIEAEISR,3,1523.370605,368.843781,-2.958420,2.788050e+03,0.0,0.0,0.000000e+00,0.000000e+00,0.0
16751911815002726321,SHC(Carbamidomethyl)IAEVEK,3,1550.230469,358.174591,4.058410,0.000000e+00,0.0,461846.0,0.000000e+00,0.000000e+00,1358150.0
1766075384941176729,SHCIAEVEK,2,1646.545044,508.247498,4.302580,2.140300e+05,0.0,104389.0,0.000000e+00,0.000000e+00,0.0
6714187641100376547,QEPERNEC(Carbamidomethyl)FLSHK,3,1717.691528,558.594849,3.741000,3.881570e+06,0.0,0.0,0.000000e+00,0.000000e+00,0.0
300941239321730683,C(Carbamidomethyl)C(Carbamidomethyl)TESLVNR,2,1726.187988,569.752625,4.410060,1.103670e+07,0.0,3691860.0,0.000000e+00,0.000000e+00,20567800.0
8470403259047476092,LC(Carbamidomethyl)VLHEK,2,1726.379639,449.744385,4.092430,4.102750e+06,0.0,760472.0,0.000000e+00,0.000000e+00,1971500.0
17001643603461665041,LC(Carbamidomethyl)VLHEK,3,1727.822021,300.165344,3.820900,1.358120e+07,0.0,0.0,0.000000e+00,0.000000e+00,12925300.0
5658659041765702685,DDSPDLPK,2,1731.364868,443.711273,4.199150,3.474620e+07,0.0,13570600.0,0.000000e+00,0.000000e+00,62024400.0
11003401133233860035,EC(Carbamidomethyl)C(Carbamidomethyl)DKPLLEK,3,1743.927124,431.205536,3.663040,0.000000e+00,0.0,2598460.0,0.000000e+00,0.000000e+00,12406600.0


In [66]:
cmap_tmt = ConsensusMapDF()

#urlretrieve ("https://raw.githubusercontent.com/OpenMS/OpenMS/develop/src/tests/topp/IsobaricAnalyzer_output_1.consensusXML", "itraq.consensusXML")
ConsensusXMLFile().load("itraq.consensusXML", cmap_tmt)
# Big data test
#ConsensusXMLFile().load("/Users/pfeuffer/Downloads/ID_mapper_merge_epi_filt_resconf_new.consensusXML", cmap_tmt)


In [67]:
df = cmap_tmt.get_intensity_df()
display(df)
print(df.info(memory_usage="deep"))

,tmt10plex_127C,tmt10plex_128C,tmt10plex_129C,tmt10plex_126,tmt10plex_130N,tmt10plex_127N,tmt10plex_129N,tmt10plex_131,tmt10plex_130C,tmt10plex_128N,file
id,,,,,,,,,,,
11812642650316679567,9477.316406,8973.786133,3362.574951,26011.720703,7224.463867,8.714253e+03,10696.969727,10535.530273,14181.990234,2864.089844,UM_F_50cm_2019_0419.mzML
5070818019611671243,7724.812988,17277.199219,5067.947754,9112.600586,0.000000,6.199816e+03,14604.040039,6825.573242,0.000000,3979.416260,UM_F_50cm_2019_0419.mzML
16110014580599801470,16066.290039,10033.250000,13595.830078,14395.139648,6765.427734,1.561013e+04,14178.379883,14921.500000,7751.592285,16088.139648,UM_F_50cm_2019_0419.mzML
1595955641419832119,9261.457031,13469.209961,9028.967773,5734.234863,4718.112305,8.227210e+03,14344.990234,9132.669922,3015.143799,8407.532227,UM_F_50cm_2019_0419.mzML
2608263041157238143,16373.799805,1553.231567,1661.919678,814.555481,5373.084961,2.813152e+03,1178.280640,1994.414307,834.245667,736.877014,UM_F_50cm_2019_0419.mzML
...,...,...,...,...,...,...,...,...,...,...,...
251517467932666866,1651.700195,2092.340820,1416.506104,9294.641602,2987.130371,3.481255e+03,2794.713623,2996.318115,3040.579102,577.065186,UM_F_50cm_2019_0418.mzML
11584007434674680918,0.000000,839.604370,913.537842,2220.770020,0.000000,1.687154e-13,0.000000,1063.503174,0.000000,0.000000,UM_F_50cm_2019_0418.mzML
14557845798613235887,2482.462402,2783.210693,490.512268,5985.435059,1244.768311,2.904880e+03,2064.895508,3355.015137,2909.781982,661.561401,UM_F_50cm_2019_0418.mzML


<class 'pandas.core.frame.DataFrame'>
UInt64Index: 737078 entries, 11812642650316679567 to 5070462102224586784
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tmt10plex_127C  737078 non-null  float32
 1   tmt10plex_128C  737078 non-null  float32
 2   tmt10plex_129C  737078 non-null  float32
 3   tmt10plex_126   737078 non-null  float32
 4   tmt10plex_130N  737078 non-null  float32
 5   tmt10plex_127N  737078 non-null  float32
 6   tmt10plex_129N  737078 non-null  float32
 7   tmt10plex_131   737078 non-null  float32
 8   tmt10plex_130C  737078 non-null  float32
 9   tmt10plex_128N  737078 non-null  float32
 10  file            737078 non-null  object 
dtypes: float32(10), object(1)
memory usage: 90.7 MB
None


In [68]:
metadf = cmap_tmt.get_metadata_df()
display(metadf)
print(metadf.info(memory_usage="deep"))

,sequence,charge,RT,mz,quality
id,,,,,
11812642650316679567,None,2,126.595421,906.788574,0.0
5070818019611671243,GILGYTEHQVVS(TMT6plex)SDFNSDTHSSTFDAGAGIALNDHF...,0,126.697800,899.849976,0.0
16110014580599801470,None,1,126.858871,403.281189,0.0
1595955641419832119,None,1,127.063797,416.786652,0.0
2608263041157238143,None,1,127.274979,594.374329,0.0
...,...,...,...,...,...
251517467932666866,None,1,9880.590820,737.632812,0.0
11584007434674680918,None,1,9898.851562,906.787415,0.0
14557845798613235887,None,0,9913.507812,737.632812,0.0


<class 'pandas.core.frame.DataFrame'>
UInt64Index: 737078 entries, 11812642650316679567 to 5070462102224586784
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   sequence  737078 non-null  object 
 1   charge    737078 non-null  int32  
 2   RT        737078 non-null  float32
 3   mz        737078 non-null  float32
 4   quality   737078 non-null  float32
dtypes: float32(3), int32(1), object(1)
memory usage: 60.4 MB
None


In [59]:
df.merge(metadf, how='left', on='id') # single table

,sequence,charge,RT,mz,quality,tmt10plex_127C,tmt10plex_128C,tmt10plex_129C,tmt10plex_126,tmt10plex_130N,tmt10plex_127N,tmt10plex_129N,tmt10plex_131,tmt10plex_130C,tmt10plex_128N,file
id,,,,,,,,,,,,,,,,
11812642650316679567,None,2,126.595421,906.788574,0.0,9477.316406,8973.786133,3362.574951,26011.720703,7224.463867,8.714253e+03,10696.969727,10535.530273,14181.990234,2864.089844,UM_F_50cm_2019_0419.mzML
5070818019611671243,GILGYTEHQVVS(TMT6plex)SDFNSDTHSSTFDAGAGIALNDHF...,0,126.697800,899.849976,0.0,7724.812988,17277.199219,5067.947754,9112.600586,0.000000,6.199816e+03,14604.040039,6825.573242,0.000000,3979.416260,UM_F_50cm_2019_0419.mzML
16110014580599801470,None,1,126.858871,403.281189,0.0,16066.290039,10033.250000,13595.830078,14395.139648,6765.427734,1.561013e+04,14178.379883,14921.500000,7751.592285,16088.139648,UM_F_50cm_2019_0419.mzML
1595955641419832119,None,1,127.063797,416.786652,0.0,9261.457031,13469.209961,9028.967773,5734.234863,4718.112305,8.227210e+03,14344.990234,9132.669922,3015.143799,8407.532227,UM_F_50cm_2019_0419.mzML
2608263041157238143,None,1,127.274979,594.374329,0.0,16373.799805,1553.231567,1661.919678,814.555481,5373.084961,2.813152e+03,1178.280640,1994.414307,834.245667,736.877014,UM_F_50cm_2019_0419.mzML
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251517467932666866,None,1,9880.590820,737.632812,0.0,1651.700195,2092.340820,1416.506104,9294.641602,2987.130371,3.481255e+03,2794.713623,2996.318115,3040.579102,577.065186,UM_F_50cm_2019_0418.mzML
11584007434674680918,None,1,9898.851562,906.787415,0.0,0.000000,839.604370,913.537842,2220.770020,0.000000,1.687154e-13,0.000000,1063.503174,0.000000,0.000000,UM_F_50cm_2019_0418.mzML
14557845798613235887,None,0,9913.507812,737.632812,0.0,2482.462402,2783.210693,490.512268,5985.435059,1244.768311,2.904880e+03,2064.895508,3355.015137,2909.781982,661.561401,UM_F_50cm_2019_0418.mzML


In [60]:
cmap_silac = ConsensusMapDF()
from urllib.request import urlretrieve
urlretrieve ("https://raw.githubusercontent.com/OpenMS/OpenMS/develop/src/tests/topp/FeatureFinderMultiplex_10_output.consensusXML", "silac.consensusXML")
ConsensusXMLFile().load("silac.consensusXML", cmap_silac)

In [61]:
display(cmap_silac.get_intensity_df())

,label 1,label 0,file
id,,,
4835329514588776807,0.0,10667330.0,FeatureFinderMultiplex_10_input.mzML
17749660155506638460,0.0,14698730.0,FeatureFinderMultiplex_10_input.mzML
7804704400743266335,0.0,466288000.0,FeatureFinderMultiplex_10_input.mzML
15004869347769368353,31916350.0,34953240.0,FeatureFinderMultiplex_10_input.mzML
3332699010107892018,5863498.0,7784946.0,FeatureFinderMultiplex_10_input.mzML
13440783915218733453,0.0,22511620.0,FeatureFinderMultiplex_10_input.mzML


In [62]:
display(cmap_silac.get_metadata_df())

,sequence,charge,RT,mz,quality
id,,,,,
4835329514588776807,None,3,2053.195312,644.968445,1.0
17749660155506638460,None,2,2053.780029,694.842407,1.0
7804704400743266335,None,2,2054.967041,683.853943,1.0
15004869347769368353,None,3,2055.814453,626.334656,1.0
3332699010107892018,None,2,2056.463623,650.868042,1.0
13440783915218733453,None,2,2058.406494,600.363159,1.0


In [63]:
cmap_silac.get_metadata_df().merge(cmap_silac.get_intensity_df(), how='left', on='id') # single table

,sequence,charge,RT,mz,quality,label 1,label 0,file
id,,,,,,,,
4835329514588776807,None,3,2053.195312,644.968445,1.0,0.0,10667330.0,FeatureFinderMultiplex_10_input.mzML
17749660155506638460,None,2,2053.780029,694.842407,1.0,0.0,14698730.0,FeatureFinderMultiplex_10_input.mzML
7804704400743266335,None,2,2054.967041,683.853943,1.0,0.0,466288000.0,FeatureFinderMultiplex_10_input.mzML
15004869347769368353,None,3,2055.814453,626.334656,1.0,31916350.0,34953240.0,FeatureFinderMultiplex_10_input.mzML
3332699010107892018,None,2,2056.463623,650.868042,1.0,5863498.0,7784946.0,FeatureFinderMultiplex_10_input.mzML
13440783915218733453,None,2,2058.406494,600.363159,1.0,0.0,22511620.0,FeatureFinderMultiplex_10_input.mzML


In [64]:
cmap_silac.get_intensity_df().index

UInt64Index([ 4835329514588776807, 17749660155506638460,  7804704400743266335,
             15004869347769368353,  3332699010107892018, 13440783915218733453],
            dtype='uint64', name='id')